In [2]:
# Initializing Pandas & Matplotlib
#import sys
#sys.path.append('/exports/reum/tdmaarseveen/modules/')
import os
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as pyplt
import io
import re
import numpy as np
import datetime
import EMR_functions as em
import Visualize as vis
from acronyms import ACRONYM_MAP
from acronyms import MEDICINE
from acronyms import TYPO_ACRONYM_MAP
from bokeh import __version__ as bv


def getcwd():
    cwd = os.getcwd()
    return cwd

def read_csv(f_name):
    return pd.read_csv(f_name, sep="|")

def remove_rtf(df_rtf, column):
    df_clean = df_rtf[~df_rtf[column].str.contains(r"^\{.*\}.*", na=False)]
    return df_clean


print('Current versions Modules:\n')
print('Pandas version:\t\t' + pd.__version__)
print('Matplotlib version:\t' + plt.__version__)
print('numpy version:\t\t' + np.__version__)
print('Bokeh version:\t\t' + bv)

Current versions Modules:

Pandas version:		0.23.0
Matplotlib version:	2.2.2
numpy version:		1.14.3
Bokeh version:		0.12.16


### 1. Load METEOR dataset

In [1]:
df_MET = pd.read_stata('METEOR_Tjardo.dta')
df_MET.head()

NameError: name 'pd' is not defined

### 2. Load matching file (HiX en Meteor)

In [3]:
import re
df_HIXMET = pd.read_excel('input_files/LUMC_Hix_nummers_METEOR.xlsx')
#print(getDim(df_HIXMET))
#
df_HIXMET['pinCode'] = df_HIXMET['pinCode'].str.replace('\.', '', regex=True)
df_HIXMET['pinCode'] = df_HIXMET['pinCode'].str.replace('[a-zA-Z]', '', regex=True)
df_HIXMET = df_HIXMET.dropna(subset=['pinCode'])
df_HIXMET.head(8)

,patientId,objectId,pinCode,SiteID
3,8797095296666,8797095296667,1554758,1
4,8797095296337,8797095296338,0865644,1
5,8797095309195,8797095309196,3156594,1
6,8797095298374,8797095298375,3888302,1
7,8797093560198,8797093560199,4929148,1
8,8797095312802,8797095312803,7664209,1
9,8797095310377,8797095310378,7847042,1
10,8797093375387,8797093375388,9301761,1


In [4]:
d_newkeys = {}

for ix, row in df_HIXMET.iterrows():
    d_newkeys[row['patientId']] = row['pinCode']
d_newkeys

{8797095296666: '  1554758',
 8797095296337: ' 0865644',
 8797095309195: ' 3156594',
 8797095298374: ' 3888302',
 8797093560198: ' 4929148',
 8797095312802: ' 7664209',
 8797095310377: ' 7847042',
 8797093375387: ' 9301761',
 8797095310896: ' 9091790',
 8797093261261: ' 9672852',
 8797094599834: '0000000',
 8797097621609: '0002693',
 8797093172698: '0003860',
 8797097436463: '0004641',
 8797097436472: '0004907',
 8797097436477: '0005161',
 8797094705836: '0006733',
 8797093449588: '0008656',
 8797100019756: '0010697',
 8797099313952: '0011329',
 8797098333600: '0011346',
 8797099313954: '0011642',
 8797099313956: '0012193',
 8797095059208: '0013818',
 8797093144522: '0013956',
 8797099313970: '0018820',
 8797093146689: '0022956',
 8797097044430: '0024402',
 8797093587135: '0024470',
 8797099314004: '0024629',
 8797097953956: '0026264',
 8797099314010: '0028253',
 8797093032659: '0028495',
 8797093039842: '0031520',
 8797097436896: '0034781',
 8797097436902: '0037940',
 8797093602924: '

### 3. Change patient ids in METEOR dataset (so it can be matched with HiX)

In [67]:
#df_MET_pat = df_MET.copy()
df_MET_pat['patientId'] = df_MET_pat['patientId'].map(d_newkeys)
#print(df_MET_pat['patientId'].unique())
#print(len(df_MET_pat[df_MET_pat['patientId'].isin(['2937513'])]))
df_MET_pat['']

28


In [15]:
df_MET_new = df_MET.copy()
df_MET_new['patientId'] = df_MET_new['patientId'].map(d_newkeys)
df_MET_new['visitDate']= pd.to_datetime(df_MET_new['minvisitdate'])+ pd.to_timedelta(df_MET_new['visitDateDif'], unit='d')

In [6]:
df_MET_new = pd.read_csv(r'featureExtraction/METEOR_subselectionFreeText2.csv', sep='|', index_col=0)
df_MET_new.head()

E:\modules\Anaconda\Program\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5,6,48,49,50,59,60,61,114,115,116,125,126,127,136,137,138,147,148,149,158,159,160,169,170,171,180,181,182,212,259,268,269,270,279,280,281,290,291,292,301,302,303,451,452,453,514,515,516,523,524,525,595,596,597,604,605,606,613,614,615,658,659,660,667,668,669,676,677,678,685,686,687,703,704,705,712,713,714,730,731,732,739,740,741,757,758,759,766,767,768,775,776,777,784,785,786,793,794,795,802,803,804,829,830,831,838,839,840,847,848,849,865,866,867,874,875,876,892,893,894,901,902,903,927,928,929,930,937,938,939,973,974,975,982,983,984,991,992,993,1027,1028,1029,1036,1037,1038,1054,1055,1056,1063,1064,1065,1072,1073,1074,1081,1082,1083,1126,1127,1128,1134,1135,1136,1137,1144,1145,1146,1161,1180,1181,1182,1189,1190,1191,1198,1199,1200,1252,1253,1254,1261,1262,1263,1315,1316,1317,1324,1325,1326,1351,1352,1353,1360,1361,1362,1369,1370,1371,1378,1379,1380,1387,1388,1389,1396,1397,1398,

,newpatientid,VisitUniqueId,patientId,Betamethasone1Name,Betamethasone1StartDate,Betamethasone1EndDate,Betamethasone1Amount,Betamethasone1Strength,Betamethasone1Dosage,Betamethasone1Iter,...,DOUBLEInfliximabsimilar,DOUBLEEtanerceptsimilar,DOUBLEvisit,DOUBLEallmediction,DOUBLEallmedictionandvisits,maxvisitdate,minvisitdate,visitDateDif,maxvisitcount,visitDate
10,8.797098e+12,8.797098e+12,5288107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,1.0,0.0,2002-04-11,2000-06-20,0.0,3.0,2000-06-20
11,8.797098e+12,8.797098e+12,5288107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,1.0,0.0,2002-04-11,2000-06-20,317.0,3.0,2001-05-03
12,8.797098e+12,8.797098e+12,5288107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,1.0,0.0,2002-04-11,2000-06-20,660.0,3.0,2002-04-11
43,8.797098e+12,8.797098e+12,8842867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,1.0,0.0,2010-12-15,2000-07-19,0.0,13.0,2000-07-19
44,8.797098e+12,8.797098e+12,8842867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,1.0,0.0,2010-12-15,2000-07-19,294.0,13.0,2001-05-09


In [28]:
print(len(df_MET))
print(len(df_MET_new[df_MET_new['patientId'].isin(list(df_REUBEL['PATNR'].unique()))]))
print(len(list(df_REUBEL[~(df_REUBEL['PATNR'].isin(l_outOfTime))]['PATNR'].unique())))

34805
27341
1127


In [27]:
#print(list(df_REUBEL[~(df_REUBEL['PATNR'].isin(l_outOfTime))]['PATNR'].unique()))

ofile = open(r'featureExtraction/PATNR_Selectie.txt', 'w')

for x in list(df_REUBEL[~(df_REUBEL['PATNR'].isin(l_outOfTime))]['PATNR'].unique()):
    ofile.write(str(x) + "\n")
ofile.close()

In [20]:
from datetime import datetime, date, timedelta
import time
l_outOfTime = df_REUBEL[(pd.to_datetime(df_REUBEL['DATUM'])<(pd.to_datetime('2011-04-29')+timedelta(days=180)))]['PATNR'].unique()
print(len(l_outOfTime))
print(len(df_REUBEL[~(df_REUBEL['PATNR'].isin(l_outOfTime))]['PATNR'].unique()))
prin

1644
1127


In [23]:
print(len(df_MET_new['patientId'].unique()))
df_RO = df_MET_new[~(df_MET_new['patientId'].isin(l_outOfTime))]
print(len(df_RO['patientId'].unique()))
df_RO.to_csv(r'featureExtraction/METEOR_subselectionFreeText2.csv', sep='|', index=True)

2203
779


In [27]:
print(df_REUBEL['PATNR'].unique())
print(list(df_MET_new['patientId'].unique()))

[9655006 7043920 4071686 ... 1999476 7475742 7614373]
['0315755', '5288107', '7193869', '8842867', '4295949', '9320663', '6779912', '7170377', '0054458', '2080488', '8524041', '6229244', '9329711', '4578040', '2342788', '1839928', '9917543', '9272242', '8249967', '3881308', '6830772', '9455892', '8058479', '7497314', '4512338', '1230764', '7348305', '1432954', '8415404', '4424316', '2156700', '5596477', '7134645', '2684371', '5201710', '4373890', '9379320', '8128990', '2728288', '5188318', '4344484', '8190696', '5076974', '3369102', '1872691', '3836089', '0224137', '9363391', '7471583', '7346232', '3386920', '3359705', '3337624', '9332853', '3319709', '6276050', '8228568', '2815407', '6394428', '2202829', '2266909', '5034097', '6803460', '5831757', '5402302', '1591638', '9974909', '1908029', '0979846', '9364532', '3353432', '6246172', '5328253', '7290604', '3311894', '9366332', '6608638', '7457283', '2937513', '2373336', '5226366', '1194233', '1170186', '7435960', '9788061', '9437763',

### 4. Load REU|Beleid & calculate Overlap between datasets

In [4]:
df_REUBEL = pd.read_csv(r'output_files/DF_REUBEL.csv', sep="|")
#df_REUBEL['PATNR'] = df_REUBEL['PATNR'].apply(str)
print('Total METEOR: ', len(df_MET['patientId'].unique()))
print('Total REUBEL: ', len(df_REUBEL['PATNR'].unique()))
print('Overlap METEOR & REUBEL: ', len(df_REUBEL[df_REUBEL['PATNR'].isin(list(df_MET_new['patientId'].unique()))]['PATNR'].unique()))

NameError: name 'df_MET' is not defined

#### 4.1 Load sample - 10 candidates

In [106]:
df_cand = pd.read_csv(r'output_files/DF_LONG_candidates.csv', index_col=0, sep="|", na_values=['NA'])
df_cand['PATNR'] = df_cand['PATNR'].apply(str)
df_cand['DATUM'] = pd.to_datetime(df_cand['DATUM'])
print('Total METEOR: ', len(df_MET['patientId'].unique()))
print('Total REUBEL: ', len(df_cand['PATNR'].unique()))
print('Overlap METEOR & REUBEL: ', len(df_cand[df_cand['PATNR'].isin(list(df_MET_new['patientId'].unique()))]['PATNR'].unique()))

Total METEOR:  4663
Total REUBEL:  10
Overlap METEOR & REUBEL:  7


In [128]:
df_match = df_MET_new[df_MET_new['patientId'].isin(list(df_cand['PATNR'].unique()))][['patientId','sex', 'yearOfBirth', 'smoking', 'RFC', 'DASMODEL', 'aCCP', 'DAS3', 'DAS4', 'DAS283', 'DAS284']] #.to_csv(r'output_files/DF_LONG_candidates_MET.csv', sep='|', index=True)

In [129]:
df_match = df_match.rename(index=str, columns={"patientId": "PATNR"}) # , 'visitDate' : 'DATUM'
df_cand = df_cand[df_cand['PATNR'].isin(list(df_MET_new['patientId'].unique()))]
#df_match['DATUM']

In [130]:
new_df = df_cand.merge(df_match, on=['PATNR'], how='left')
new_df = new_df.drop_duplicates(subset=['PATNR', 'DATUM', 'drug', 'actie'])
new_df

,PATNR,DATUM,Bezdat0,drug,actie,XANTWOORD,dosering,sex,yearOfBirth,smoking,RFC,DASMODEL,aCCP,DAS3,DAS4,DAS283,DAS284
0,2232135,2015-07-13,2015-07-13 00:00:00+00:00,NaN,NaN,eac ^controle aanstaande vrijdag bij collega d...,,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51
10,2232135,2015-07-17,2015-07-13 00:00:00+00:00,methotrexaat,start,start methotrexaat ^ uitleg : ^ - vaste dag pe...,,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51
20,2232135,2015-07-17,2015-07-13 00:00:00+00:00,prednison,start,start methotrexaat ^ uitleg : ^ - vaste dag pe...,120 mg,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51
30,2232135,2015-08-24,2015-07-13 00:00:00+00:00,methotrexaat,door,iom jvr : ^ - methotrexaat 15 mg / week door ^...,15 mg,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51
40,2232135,2015-11-16,2015-07-13 00:00:00+00:00,methotrexaat,stop,iom fga ^b / ^stop methotrexaat en fz ^start s...,,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51
50,2232135,2015-11-16,2015-07-13 00:00:00+00:00,sulfasalazine,start,iom fga ^b / ^stop methotrexaat en fz ^start s...,1000 mg,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51
60,2232135,2015-11-16,2015-07-13 00:00:00+00:00,depomedrol,start,iom fga ^b / ^stop methotrexaat en fz ^start s...,120 mg,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51
70,2232135,2015-12-24,2015-07-13 00:00:00+00:00,NaN,NaN,bespreken in pb : ^ - leflunomide ? maar wel b...,NaN,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51
80,2232135,2016-01-07,2015-07-13 00:00:00+00:00,methotrexaat,start,eerst lokale injectie polsen en methotrexaat s...,15 mg,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51
90,2232135,2016-01-18,2015-07-13 00:00:00+00:00,NaN,NaN,"co inplannen voor lokale injectie polsen , dan...",,0.0,1950.0,NaN,1.0,53.0,1.0,2.55,2.47,4.73,4.51


In [121]:
new_df.sort_values(['PATNR', 'DATUM']).to_csv(r'featureExtraction/DF_LONG_candidates_withMETEOR.csv', sep='|', index=True)

In [137]:
print(df_MET_new[df_MET_new['patientId']=='2232135'][['DAS4', 'visitDate']])
print(df_MET_new[df_MET_new['patientId']=='2232135'][['DAS3', 'visitDate']])
print(df_MET_new[df_MET_new['patientId']=='2232135'][['DAS284', 'visitDate']])
print(df_MET_new[df_MET_new['patientId']=='2232135'][['DAS283', 'visitDate']])

       DAS4  visitDate
32910  2.47 2015-07-12
32911  2.83 2015-07-16
32912   NaN 2015-08-23
32913  2.87 2015-11-15
32914   NaN 2016-02-03
32915  2.48 2016-03-20
32916  2.06 2016-06-30
32917  2.01 2016-11-10
32918  2.14 2017-03-02
32919  1.98 2017-06-29
       DAS3  visitDate
32910  2.55 2015-07-12
32911  2.56 2015-07-16
32912   NaN 2015-08-23
32913  2.60 2015-11-15
32914   NaN 2016-02-03
32915  2.42 2016-03-20
32916  2.07 2016-06-30
32917  2.16 2016-11-10
32918  2.15 2017-03-02
32919  1.99 2017-06-29
       DAS284  visitDate
32910    4.51 2015-07-12
32911    5.24 2015-07-16
32912     NaN 2015-08-23
32913    5.15 2015-11-15
32914     NaN 2016-02-03
32915    4.78 2016-03-20
32916    3.90 2016-06-30
32917    3.57 2016-11-10
32918    3.68 2017-03-02
32919    3.73 2017-06-29
       DAS283  visitDate
32910    4.73 2015-07-12
32911    4.74 2015-07-16
32912     NaN 2015-08-23
32913    4.65 2015-11-15
32914     NaN 2016-02-03
32915    4.71 2016-03-20
32916    3.91 2016-06-30
32917    3.86 2016-

In [124]:
print(df_MET_new['DASMODEL'].unique())

[53. 28. nan]


In [126]:
print(df_MET_new['DAS3'].unique())

[3.6  3.01 1.06 1.65 1.93 1.35  nan 2.44 3.34 1.3  2.92 0.99 1.45 1.39
 2.16 2.14 1.9  3.08 2.38 2.77 3.94 0.76 2.23 1.02 2.41 1.7  1.92 2.79
 2.55 2.56 2.53 1.22 1.58 1.01 1.08 0.93 1.49 0.82 1.68 1.19 2.72 0.72
 1.13 3.15 3.31 1.71 1.2  1.56 1.36 0.45 1.53 2.61 1.78 0.95 1.63 2.3
 2.09 1.29 2.22 1.74 1.16 1.09 2.28 2.13 2.52 2.24 3.12 2.36 2.05 2.68
 2.5  2.63 2.15 1.85 2.43 2.25 1.81 2.2  1.5  3.19 2.54 3.43 2.98 2.47
 1.96 2.66 2.34 1.41 4.31 2.08 2.64 3.23 2.37 2.76 2.71 2.1  2.69 3.65
 3.71 2.62 3.91 4.42 3.1  5.91 5.59 2.89 5.05 3.05 1.98 2.88 1.86 2.39
 2.03 1.99 1.95 1.91 1.69 1.4  2.59 2.7  1.64 2.9  2.07 1.47 2.04 1.32
 2.49 1.83 2.82 2.73 1.72 3.99 2.12 2.32 3.98 3.48 2.67 3.69 2.29 2.11
 2.57 3.13 2.33 3.17 2.17 2.91 2.26 1.88 2.27 1.59 1.52 3.06 1.76 1.87
 1.62 2.21 2.02 4.24 3.5  2.19 2.99 5.69 1.12 1.75 1.28 3.2  4.18 3.67
 3.51 2.94 1.48 2.84 2.42 3.36 3.89 2.01 1.14 3.03 1.61 1.94 1.24 1.34
 1.43 1.31 4.55 4.26 1.67 4.05 2.48 2.4  1.38 2.8  1.66 2.35 5.2  5.
 2.85 3.6

### Make new DF_REUBEL -> with patients occuring in METEOR & Recently added 

In [7]:
# 'visitDate' meenemen? 
df_subREU = df_REUBEL[df_REUBEL['PATNR'].isin(list(df_MET_new['patientId'].unique()))]
df_subREU.to_csv(r'featureExtraction/DF_OVERLAP_METEOR_HIX.csv', sep='|', index=True)
#df_subREU = 

In [9]:
print(len(df_subREU))
print(len(df_REUBEL))

10492
45012
